In [6]:
!pip install --quiet arcgis

In [36]:
import os
from arcgis.gis import GIS;
from arcgis.mapping import WebMap;
from arcgis.features import SpatialDataFrame
import pandas as pd;
import arcpy;


In [37]:
fp_path  = r"F:\SitingStagingCache";
fp_fgdb  = r"footprint.gdb";
fp_full  = fp_path + os.sep + fp_fgdb;
fp_layer = "NED13";
fp_trg   = fp_path + os.sep + "NED13" + os.sep + "trg";

resources = [f.name for f in os.scandir(fp_trg) if f.is_dir()];

len(resources)

1363

In [38]:
if not os.path.exists(fp_full):
    arcpy.CreateFileGDB_management(fp_path,fp_fgdb);
    
if arcpy.Exists(fp_full + os.sep + fp_layer):
    arcpy.Delete_management(fp_full + os.sep + fp_layer);
    
arcpy.CreateFeatureclass_management(
     out_path          = fp_full
    ,out_name          = fp_layer
    ,geometry_type     = "POLYGON"
    ,has_m             = "DISABLED"
    ,has_z             = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269)
    ,config_keyword    = None
);

arcpy.env.workspace = fp_full;
arcpy.management.AddFields(
     fp_layer
    ,[
         ['resource_name' ,'TEXT' ,'Resource_Name'  ,255, None,'']
        ,['resource_path' ,'TEXT' ,'Resource_Path'  ,255, None,'']
     ]
);
    

In [39]:
cursor = arcpy.da.InsertCursor(
     fp_full + os.sep + fp_layer
    ,["resource_name","resource_path","SHAPE@"]
);

for item in resources:
    
    if len(item) == 7:
        img = 'img' + item + '_13.img';
    else:
        img = item + '.img';
        
    raster = fp_trg + os.sep + item + os.sep + img;
    
    ar = arcpy.Raster(raster);
    ll = getattr(ar.extent,"lowerLeft");
    lr = getattr(ar.extent,"lowerRight");
    ur = getattr(ar.extent,"upperRight");
    ul = getattr(ar.extent,"upperLeft");
    
    rg = arcpy.Array([
         arcpy.Point(ul.X,ul.Y)
        ,arcpy.Point(ll.X,ll.Y)
        ,arcpy.Point(lr.X,lr.Y)
        ,arcpy.Point(ur.X,ur.Y)
        ,arcpy.Point(ul.X,ul.Y)
    ]);
    
    fp = arcpy.Polygon(rg,ar.spatialReference);
    
    cursor.insertRow([img,raster,fp]);

if cursor:
    del cursor;
    
'footprints generated'

'footprints generated'

In [40]:
gis = GIS();
sdf = pd.DataFrame.spatial.from_featureclass(fp_full + os.sep + fp_layer);

wmap = gis.map("North America");
sdf.spatial.plot(
     kind = 'map'
    ,map_widget = wmap
    ,symbol_type='simple'
    ,colors = 'Greens_r'
    ,outline_color = 'Dark2_r'
    ,line_width = 0.5
);
    
wmap
    

MapView(layout=Layout(height='400px', width='100%'))